In [3]:
import numpy as np
import os
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

In [71]:
common_data_path = 'Data/USCensusACS/IncomeAndOccupancySeattle/'
# Read in the census data, using the GEO.id column as the index
df2012 = pd.read_csv(os.path.join(common_data_path, 'ACS_12_1YR_B25118_with_ann.csv'),
                    index_col='GEO.id')
print(df2012)

df2013 = pd.read_csv(os.path.join(common_data_path, 'ACS_13_1YR_B25118_with_ann.csv'),
                    index_col='GEO.id')
print(df2012)

df2014 = pd.read_csv(os.path.join(common_data_path, 'ACS_14_1YR_B25118_with_ann.csv'),
                    index_col='GEO.id')

                  GEO.id2         GEO.display-label         HD01_VD01  \
GEO.id                                                                  
Id                    Id2                 Geography  Estimate; Total:   
0400000US53            53                Washington           2636817   
1600000US5363000  5363000  Seattle city, Washington            289790   

                                HD02_VD01                  HD01_VD02  \
GEO.id                                                                 
Id                Margin of Error; Total:  Estimate; Owner occupied:   
0400000US53                         10074                    1643637   
1600000US5363000                     4363                     132888   

                                         HD02_VD02  \
GEO.id                                               
Id                Margin of Error; Owner occupied:   
0400000US53                                  12066   
1600000US5363000                              3805   

  

In [38]:
print(df2012['HD01_VD03'])

GEO.id
Id                  Estimate; Owner occupied: - Less than $5,000
0400000US53                                                25022
1600000US5363000                                            2069
Name: HD01_VD03, dtype: object


In [6]:
type(df2012)

pandas.core.frame.DataFrame

In [39]:
df2012.index

Index(['Id', '0400000US53', '1600000US5363000'], dtype='object', name='GEO.id')

In [40]:
seattle2012 = df2012.loc['1600000US5363000']
print(seattle2012[['HD01_VD01']])
type(seattle2012)

HD01_VD01    289790
Name: 1600000US5363000, dtype: object


pandas.core.series.Series

In [66]:
def income_bracket_buckets(df):
    """ Return two Series of Seattle renters by income bracket, and owners by income bracket """

    seattle_row = df.loc['1600000US5363000']
    #print("ibb: seattle_row={0}".format(seattle_row))
    #print("ibb: read csv default type = {0}".format(type(seattle_row.HD01_VD13)))
    
    ##
    # Owner
    seattle_owner = Series([0] * 5, 
                           index=['>$150K', '$100K-$150K', '$75K-$100K', '$50K-$75K', '<$50K'])
    seattle_owner['>$150K'] = int(seattle_row.HD01_VD13)
    seattle_owner['$100K-$150K'] = int(seattle_row.HD01_VD12)
    seattle_owner['$75K-$100K'] = int(seattle_row.HD01_VD11)
    seattle_owner['$50K-$75K'] = int(seattle_row.HD01_VD10)
    # Under 50K has multiple sub-buckets that will be combined into one here
    owner_under50K_cols = ['HD01_VD03', 'HD01_VD04', 'HD01_VD05', 'HD01_VD06', 
                         'HD01_VD07', 'HD01_VD08', 'HD01_VD09']
    # Convert <50K cells from string to integer before summing
    seattle_row[owner_under50K_cols] = seattle_row[owner_under50K_cols].astype('int')
    seattle_owner['<$50K'] = seattle_row[owner_under50K_cols].values.sum()
    #print(seattle_owner)
    
    ##
    # Renter
    seattle_renter = Series([0] * 5, 
                           index=['>$150K', '$100K-$150K', '$75K-$100K', '$50K-$75K', '<$50K'])
    seattle_renter['>$150K'] = int(seattle_row.HD01_VD25)
    seattle_renter['$100K-$150K'] = int(seattle_row.HD01_VD24)
    seattle_renter['$75K-$100K'] = int(seattle_row.HD01_VD23)
    seattle_renter['$50K-$75K'] = int(seattle_row.HD01_VD22)
    # Under 50K has multiple sub-buckets that will be combined into one here
    renter_under50K_cols = ['HD01_VD15', 'HD01_VD16', 'HD01_VD17', 'HD01_VD18', 
                         'HD01_VD19', 'HD01_VD20', 'HD01_VD21']
    # Convert <50K cells from string to integer before summing
    seattle_row[renter_under50K_cols] = seattle_row[renter_under50K_cols].astype('int')
    seattle_renter['<$50K'] = seattle_row[renter_under50K_cols].values.sum()    
    return (seattle_owner, seattle_renter)



In [73]:
owner2012, renter2012 = income_bracket_buckets(df2012)
owner2013, renter2013 = income_bracket_buckets(df2013)
owner2014, renter2014 = income_bracket_buckets(df2014)

print("Owner2012:\n{0}".format(owner2012))
print("Owner2013:\n{0}".format(owner2013))
print("Owner2014:\n{0}".format(owner2014))
print("Renter2012:\n{0}".format(renter2012))
print("Renter2013:\n{0}".format(renter2013))
print("Renter2014:\n{0}".format(renter2014))

Owner2012:
>$150K         35727
$100K-$150K    27618
$75K-$100K     18994
$50K-$75K      21803
<$50K          28746
dtype: int64
Owner2013:
>$150K         41538
$100K-$150K    29966
$75K-$100K     17483
$50K-$75K      21317
<$50K          27436
dtype: int64
Owner2014:
>$150K         47471
$100K-$150K    29107
$75K-$100K     16652
$50K-$75K      19346
<$50K          26062
dtype: int64
Renter2012:
>$150K          8096
$100K-$150K    17836
$75K-$100K     17701
$50K-$75K      28030
<$50K          85239
dtype: int64
Renter2013:
>$150K         12824
$100K-$150K    19025
$75K-$100K     18153
$50K-$75K      27253
<$50K          82925
dtype: int64
Renter2014:
>$150K         14084
$100K-$150K    20883
$75K-$100K     17325
$50K-$75K      26894
<$50K          86740
dtype: int64


In [74]:
# Merge tables into dataframe
seattleOwnerRenter = DataFrame({'Owner2012': owner2012,
                                'Owner2013': owner2013,
                                'Owner2014': owner2014,
                                'Renter2012': renter2012,
                                'Renter2013': renter2013,
                                'Renter2014': renter2014})
print(seattleOwnerRenter)

             Owner2012  Owner2013  Owner2014  Renter2012  Renter2013  \
>$150K           35727      41538      47471        8096       12824   
$100K-$150K      27618      29966      29107       17836       19025   
$75K-$100K       18994      17483      16652       17701       18153   
$50K-$75K        21803      21317      19346       28030       27253   
<$50K            28746      27436      26062       85239       82925   

             Renter2014  
>$150K            14084  
$100K-$150K       20883  
$75K-$100K        17325  
$50K-$75K         26894  
<$50K             86740  
